In [1]:
import os
import pandas as pd
import numpy as np
from scipy import stats
from pathlib import Path
from dotenv import load_dotenv

from src.metrics import pearson_metric

load_dotenv()

True

In [2]:
dataset_dir = Path(os.environ['dataset_dir'])
data = pd.read_csv(dataset_dir / 'train.csv')

In [3]:
data = data.set_index('row_id')

In [4]:
# train on earlier data, test on later data
train = data[data.time_id < 1000]
val = data.query("1000 <= time_id < 1100")
test = data.query("1100 <= time_id")

x_train, y_train = train.drop('target', axis=1), train[['time_id', 'target']]
x_val, y_val = val.drop('target', axis=1), val[['time_id', 'target']]
x_test, y_test = test.drop('target', axis=1), test[['time_id', 'target']]

In [5]:
class PearsonMetric(object):
    def get_final_error(self, error, weight):
        return error

    def is_max_optimal(self):
        return True

    def evaluate(self, approxes, target, weight):
        # костыль, но я не знаю что с ним делать,
        # катбуст такие штуки не поддерживает
        if len(target) == len(y_val.target):
            target = y_val
        elif len(target) == len(y_train.target):
            target = y_train
        
        y_pred = pd.DataFrame({'target': approxes[0]}, index=target.index)
        coef = pearson_metric(target, y_pred)
        return coef, 0


In [6]:
from catboost import CatBoostRegressor, Pool

train_pool = Pool(x_train.drop('time_id', axis=1), y_train.target, cat_features=['investment_id'])
val_pool = Pool(x_val.drop('time_id', axis=1), y_val.target, cat_features=['investment_id'])

model = CatBoostRegressor(
    iterations=400,
    random_state=0,
    cat_features=['investment_id'], 
    verbose=40,
    eval_metric=PearsonMetric(),
)

In [7]:
model = model.fit(train_pool, eval_set=val_pool)

Learning rate set to 0.30288


/miniconda/envs/crystalvae/lib/python3.6/site-packages/catboost/core.py:1464: UserWarning: Failed to optimize method "evaluate" in the passed object:
Failed in nopython mode pipeline (step: nopython frontend)
Untyped global name 'y_val': Cannot determine Numba type of <class 'pandas.core.frame.DataFrame'>

File "<ipython-input-5-4cb548f2dab3>", line 9:
    def evaluate(self, approxes, target, weight):
        if len(target) == len(y_val.target):
        ^

  self._object._train(train_pool, test_pool, params, allow_clear_pool, init_model._object if init_model else None)


0:	learn: -0.0005814	test: 0.0758589	best: 0.0758589 (0)	total: 5.15s	remaining: 34m 16s
40:	learn: 0.0005089	test: 0.1216440	best: 0.1216440 (40)	total: 3m 8s	remaining: 27m 34s
80:	learn: 0.0004282	test: 0.1247225	best: 0.1247225 (80)	total: 6m 7s	remaining: 24m 9s
120:	learn: 0.0003989	test: 0.1225032	best: 0.1272309 (106)	total: 9m 3s	remaining: 20m 52s
160:	learn: 0.0002786	test: 0.1177123	best: 0.1272309 (106)	total: 11m 53s	remaining: 17m 38s
200:	learn: 0.0001373	test: 0.1186822	best: 0.1272309 (106)	total: 14m 46s	remaining: 14m 37s
240:	learn: 0.0000921	test: 0.1172115	best: 0.1272309 (106)	total: 17m 40s	remaining: 11m 39s
280:	learn: 0.0001314	test: 0.1155979	best: 0.1272309 (106)	total: 20m 30s	remaining: 8m 41s
320:	learn: 0.0003007	test: 0.1144597	best: 0.1272309 (106)	total: 23m 22s	remaining: 5m 45s
360:	learn: 0.0002947	test: 0.1146621	best: 0.1272309 (106)	total: 26m 15s	remaining: 2m 50s
399:	learn: 0.0001400	test: 0.1151824	best: 0.1272309 (106)	total: 29m	remainin

In [10]:
test_pool = Pool(x_test, cat_features=['investment_id'])
y_pred = model.predict(test_pool)
y_pred = pd.DataFrame({'target': y_pred}, index=x_test.index)

In [11]:
pearson_metric(y_test, y_pred)

0.10990725295039389

In [13]:
weights_dir = dataset_dir / 'weights'
os.makedirs(weights_dir, exist_ok=True)
model.save_model(weights_dir / 'cb_first_test_109')